## Chuyển đổi sang Ollama - Few-Shot

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import requests
import json

In [ ]:
# Thay 'llama3' bằng tên model Ollama bạn muốn sử dụng (ví dụ: 'mistral', 'llama2', v.v.)
ollama_model = 'llama3'
domain_list = ['mobile', 'fashion', 'cosmetic', 'app']

In [ ]:
def call_ollama(prompt):
    """Hàm gửi yêu cầu đến Ollama API và nhận kết quả."""
    url = 'http://localhost:11434/api/generate'
    payload = {
        "model": ollama_model,
        "prompt": prompt,
        "stream": False
    }
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()  # Báo lỗi nếu request không thành công
        response_data = response.json()
        return response_data.get('response', '').strip()
    except requests.exceptions.RequestException as e:
        print(f"Lỗi khi gọi Ollama API: {e}")
        return ""

for domain in domain_list:
  datafile = domain
  # Bạn cần đảm bảo đường dẫn đến file là chính xác
  df_train = pd.read_excel(f'/content/viocd_train_{datafile}.xlsx')
  df_test = pd.read_excel(f'/content/viocd_test_{datafile}.xlsx')
  df_test['Prediction'] = ''

  # Create a TfidfVectorizer
  tfidf_vectorizer = TfidfVectorizer()
  tfidf_matrix = tfidf_vectorizer.fit_transform(df_train['Comment'])
  # Get indices of comments with label 0,1
  indices_label_0 = df_train[df_train['Label'] == 0].index
  indices_label_1 = df_train[df_train['Label'] == 1].index

  def get_similarity_comments(text_input):
    text_input_tfidf = tfidf_vectorizer.transform([text_input])
    cosine_similarities = cosine_similarity(text_input_tfidf, tfidf_matrix).flatten()
    similarities_label_0 = cosine_similarities[indices_label_0]
    similarities_label_1 = cosine_similarities[indices_label_1]
    top_indices_label_0 = indices_label_0[similarities_label_0.argsort()[-1:][::-1]]
    top_comments_label_0 = df_train.loc[top_indices_label_0]['Comment'].tolist()
    top_indices_label_1 = indices_label_1[similarities_label_1.argsort()[-1:][::-1]]
    top_comments_label_1 = df_train.loc[top_indices_label_1]['Comment'].tolist()
    return top_comments_label_0, top_comments_label_1

  def create_instruction_prompt(example_comments_label_0, example_comments_label_1):
    identity = "# Identity\nBạn là chuyên gia phân loại văn bản, hãy gắn nhãn các bình luận thương mại điện tử có sự phàn nàn là '1', ngược lại là '0'."
    instructions = "# Instructions\nCâu trả lời có một từ duy nhất, là '1' hoăc '0' và không có bình luận bổ sung."
    examples = f"# Examples\n<review id='example-1'>\n{example_comments_label_0[0]}\n</review>\n<response id='example-1'>\n0\n</response>\n<review id='example-2'>\n{example_comments_label_1[0]}\n</review>\n<response id='example-2'>\n1\n</response>"
    return f"{identity}\n{instructions}\n{examples}"

  for index, row in df_test.iterrows():
    comment = row['Comment']
    top_comments_label_0, top_comments_label_1 = get_similarity_comments(comment)
    
    # Tạo phần đầu của prompt với các ví dụ
    instruction_prompt = create_instruction_prompt(top_comments_label_0, top_comments_label_1)
    
    # Tạo prompt hoàn chỉnh cho Ollama
    full_prompt = f"{instruction_prompt}\n<review id='input'>\n{comment}\n</review>\n<response id='input'>"

    # Gọi Ollama API
    prediction = call_ollama(full_prompt)
    
    df_test.loc[index, 'Prediction'] = prediction
    print(f"Đã xử lý comment {index + 1}/{len(df_test)} của domain {domain}")

  df_test.to_excel(f'/content/viocd_prediction_{datafile}_fews_{ollama_model}.xlsx', index=False)
  print(f"Đã lưu kết quả cho domain: {domain}")

In [ ]:
# Đoạn code này để đánh giá kết quả, bạn có thể chạy sau khi có file prediction
from sklearn.metrics import classification_report
df_result = pd.read_excel(f'/content/viocd_prediction_mobile_fews_{ollama_model}.xlsx') # Thay 'mobile' bằng domain bạn muốn xem
y_true = [str(label) for label in df_result['Label']]
y_pred = [str(label) for label in df_result['Prediction']]
report = classification_report(y_true, y_pred, labels=['0', '1'])
print(report)